In [ ]:
# Imports
import pandas as pd
import numpy as np
import duckdb

In [ ]:
# Load data
df1 = duckdb.sql("""
    SELECT *
    FROM read_csv_auto('dataset/dnc_list_final.csv')
""").df()

df2 = duckdb.sql("""
    SELECT *
    FROM read_csv_auto('dataset/RAW/JAS233_JBT.csv', normalize_names=True)
""").df()

In [ ]:
# Convert phone to int
df2['work_phone'] = df2['work_phone'].astype(np.int64)

In [ ]:
# Checks if Phone from RAW file is in DNC list
is_dnc = duckdb.sql("""
    SELECT DISTINCT a.*,
        CASE
            WHEN b.value IS NOT NULL THEN 'Yes' ELSE 'No'
        END AS is_dnc
    FROM df2 a
    LEFT JOIN df1 b on a.work_phone = b.value
""").df()

is_dnc.tail(3)

In [ ]:
# Export
duckdb.sql("""
    COPY is_dnc TO 'output/jas233_jbt_cleaned.csv' (HEADER 1, DELIMITER ',')
""")